In [44]:
# TODOs: Operate DB Class
# TODOs: Use SQLite
# TODOs: Implement BM25

from elasticsearch import Elasticsearch

# INIT OBJECT

PORT = "http://localhost:9200"
INDEX_NAME = "knowledge"
errors_before_interrupt = 5
refresh_index_after_insert = False
max_insert_retries = 3
yield_ok = False

wiki_ev = Elasticsearch(
    PORT,
    #http_auth=(es_api_user, es_api_password)
    retry_on_timeout=True,  # should timeout trigger a retry on different node?
)

wiki_ev.elastic_index = INDEX_NAME

wiki_ev

<Elasticsearch(['http://localhost:9200'])>

In [49]:
### SEARCH FUNCTION ###

# TODOs: Implement as Class-DB object

def search_text(es, query_, k=5):
    results = es.search(
        index = es.elastic_index,
        query = {
            "match": {
                "document.text": query_,
                },
        },
        size=k)

    hits = results["hits"]["hits"]
    doc_ids = [row['_source']["document"]["id"] for row in hits]

    return hits

In [86]:
### LOAD DATA ###

import json

data = [json.loads(ln) for ln in open("../data/train_cmv_cleaned.jsonl")]
kp = [json.loads(ln) for ln in open("../data/keyphrases.jsonl")]
ex_output = [json.loads(ln) for ln in open("../data/wiki_doc_retrieved_from_op_train.jsonlist")]
conan = [json.loads(ln) for ln in open("../data/CONAN.json")]


In [80]:
### SAMPLE ###

sample_args = data[0:5]
sample_kp = kp[0:5]
sample_output = ex_output[0:5]

In [48]:
### SAMPLE OUTPUT ###

_ = 2
sample = sample_output[_]["retrieval_results"]

print(len(sample))
sample


6


[{'retrieved_document_titles': ['Presidential system',
   'Turkish general election, June 2015',
   'Cabinet (government)',
   'Parliamentary system',
   'Politics of Russia'],
  'query': 'the executive the presidential system legislative branches partisan unproductive',
  'sentence': 'My View : I think the presidential system leads to a conflict between the executive and legislative branches of government that is highly partisan and unproductive . '},
 {'retrieved_document_titles': ['NSA warrantless surveillance (2001–07)',
   'United States Intelligence Community Oversight',
   'Separation of powers in Singapore',
   'Government of Oklahoma',
   'Hein v. Freedom From Religion Foundation'],
  'query': 'executive branch the judiciary the judiciary the executive oversight regards legislative legislative',
  'sentence': 'I understand the separation of powers with regards to ensuring the independence of the judiciary but separating the executive and legislative seems unnecessary as the ju

In [96]:
import keyphrase_extraction
from keyphrase_extraction import exctract_keyphrase
import re

# TODOs: Ranking
# TODOs: Stance
# TODOs: Paralellise


sample_hate = conan[0:100]

_ = 85
#ar_ = sample_args[_]["arguments"]
hate_ = sample_hate[0]["conan"][_]["hateSpeech"]

queries = []
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', hate_)

hate_

# results = []

# for i in sentences:
#     # TODOs: Refactor key_BERT as Class for multiple oututs
#     kp_ = (exctract_keyphrase(i, n_gram=1, n_kp=1), exctract_keyphrase(i, n_gram=2, n_kp=1), exctract_keyphrase(i, n_gram=3, n_kp=1))
    
#     query = ", ".join(str(i[0]) for i in kp_)
#     res = [i["_source"]["document"]["title"] for i in search_text(es=wiki_ev, query_=query, k=5)]

#     results.append({
#         "argument_sentence": i, 
#         "query": query, 
#         "retireved_documents": res,

#     })

# results

'I am done with Islam and isis. All Muslims should be sent to their homeland. Britain will be better without their violence and ideology.'

In [ ]:
#queries
query = "the executive the presidential system legislative branches partisan unproductive"
res = search_text(wiki_ev, query_=query, k=3)
res

[{'_index': 'knowledge',
  '_id': 'kbZa5YIBRqC1428DaQUk',
  '_score': 33.672318,
  '_ignored': ['document.text.keyword'],
  '_source': {'document': {'id': '3408791',
    'source': 'wikipedia',
    'title': 'Ruling party',
    'text': "In parliamentary systems, the majority in the legislature also controls the executive branch of government, thus leaving no possibility of opposing parties concurrently occupying the executive and legislative branches of government. In other systems, such as in an American style presidential system, the party of the president does not necessarily also have a legislative majority. A 'ruling party' is also used to describe the party of one-party states, such as the Chinese Communist Party in the People's Republic of China. In his political manifesto 'The Green Book', the late Libyan leader Muammar al-Gaddafi attacked the ability of the ruling party, using it as a basis for his opposition to partisan politics. None"}}},
 {'_index': 'knowledge',
  '_id': 'JQ3

In [ ]:
import re

print(", ".join(re.sub(r'\[\[(?:[^|\]]*\|)?([^\]]*)]]', "", i["_source"]["document"]["title"]).strip("[]") for i in res))

Ruling party, Fusion of powers, Student governments in the United States


In [ ]:
### TEST SEARCH ###

def search_text(es, query_, k=5):
    results = es.search(
        index = es.elastic_index,
        query = {
            "match": {
                "document.text": query_,
                },
        },
        size=k)

    hits = results["hits"]["hits"]
    doc_ids = [row['_source']["document"]["id"] for row in hits]

    return hits

def search_topic(es, topic, k=5):
    results = es.search(
        index = es.elastic_index,
        body= {
            "size": k,
            "query": {
                "match": {
                    "document.title": topic,
        }}})

    hits = results["hits"]["hits"]
    doc_ids = [row['_source']["document"]["id"] for row in hits]

    title = hits[0]["_source"]["document"]["title"]
    text = hits[0]["_source"]["document"]["text"]

    return {
        "title": title,
        "text": text
    }

query = "government emails"
text = search_text(wiki_ev, query_=query, k=10)

print(text)


In [1]:
from datasets import load_dataset

dataset = load_dataset("cc_news")


Reusing dataset cc_news (/Users/joshua.sheppard/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/ae469e556251e6e7e20a789f93803c7de19d0c4311b6854ab072fecb4e401bd6)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
cc_news_data = [i for i in dataset["train"]]
len(cc_news_data)

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url'],
        num_rows: 708241
    })
})

In [9]:
def iter_data(dataset):
    for i in dataset["train"]:
        yield i

cc_news_data = iter_data(dataset=dataset)

fout = open("../data/cc_news.jsonl", "w")
for ln in cc_news_data:
    fout.write(json.dumps({
        "title": ln["title"],
        "text": ln["text"],
        "domain": ln["domain"],
        "date": ln["date"], 
        "url": ln["url"],
    }))
    
    fout.write("\n")

fout.close


TypeError: write() argument must be str, not dict

<generator object iter_data at 0x3fbe52420>